In [15]:
import networkx as nx
import pandas as pd
import numpy as np
import random 
import scipy.stats as stats
import matplotlib.pyplot as plt
import sys, os
import warnings
import processing_module as pm
import scipy.stats as stats
from tqdm import tqdm

warnings.filterwarnings("ignore")

<span style="color:orange">
    
## Preferential attachment ##

В этой модели новый узел присоединяется к уже существующему узлу $i$ в ненаправленном графе с вероятностью, вычисляемой по формуле:

\begin{equation}\label{F1}
P_{PA}(i,t) = \dfrac{d_{i,t}}{\sum\limits_{s \in V_{t}} d_{s,t}},
\end{equation}

где $d_{i,t}$ - это количество связей узла $i$, принадлежащего множеству узлов $V_{t}$ в момент времени $t$. 

In [2]:
data = {
    'Source': [1, 2],
    'Target': [2, 1],
    'Node_degree': [1, 1]
}

row_labels = [1, 2]

PA_graph = pd.DataFrame(data=data, index=row_labels)

In [3]:
print(PA_graph)

   Source  Target  Node_degree
1       1       2            1
2       2       1            1


In [4]:
print(PA_graph['Source'].values)

[1 2]


In [5]:
print(PA_graph['Node_degree'].values)

[1 1]


In [18]:
flg_PA = 0

In [19]:
l = []
if(flg_PA == 1):
    for i in tqdm(range(3, 10000)):
        k = (random.choices(PA_graph['Source'].values, weights=PA_graph['Node_degree'].values))[0]
        if (i == 3):
            l = [PA_graph['Source'].values, PA_graph['Node_degree'].values]
        # print(k)
        PA_graph.loc[i] = [i, k, 1]
        PA_graph['Node_degree'][k] += 1

print(l)

[]


In [7]:
print(PA_graph)

   Source  Target  Node_degree
1       1       2            1
2       2       1            1


In [8]:
# PA_graph[['Source', 'Target']].to_csv('output.csv')

In [9]:
# print(sorted(PA_graph['Node_degree'], reverse=True))

<span style="color:orange">
    
## Clustering attachment ##

The attachment to an existing node i is proceeded with probability proportional to its clustering coefficient:

PCA(i) ∝ cαi + ϵ, 

ci = 2∆i / ki(ki − 1)


In [2]:
data_ca = {
    'Source': [1, 2, 3],
    'Target': [2, 3, 1],
}

row_labels = [1, 2, 3]

CA_graph = pd.DataFrame(data=data_ca, index=row_labels)


In [3]:
print(CA_graph)

   Source  Target
1       1       2
2       2       3
3       3       1


In [4]:
G_ca = nx.Graph()
G_ca.add_edges_from(CA_graph.values.tolist())

In [5]:
print(nx.clustering(G_ca).keys())
print(list(nx.clustering(G_ca).values()))

dict_keys([1, 2, 3])
[1.0, 1.0, 1.0]


In [20]:
flg_CA = 0

In [21]:
if(flg_CA == 1):
    for i in tqdm(range(4, 10000)):
        local_calst = list(nx.clustering(G_ca).values())
        for j in range(len(local_calst)):
            local_calst[j] += 0.001
        k = (random.choices(list(nx.clustering(G_ca).keys()), weights=local_calst))[0]
        G_ca.add_edge(k, i)


In [11]:
print(G_ca)

Graph with 9621 nodes and 9621 edges


In [22]:
my_path = os.path.abspath("") + "/.."
(nx.to_pandas_edgelist(G_ca)).to_csv(my_path + "/Data_postproc" + '/CA_graph.csv')

In [28]:
edges_ca = nx.to_pandas_edgelist(G_ca)
print(edges_ca.groupby(['source']).count().sort_values(by = ['target'] , ascending=False))

        target
source        
1           40
3           34
2           34
26          12
126         12
...        ...
1177         1
3652         1
3653         1
3656         1
9465         1

[4853 rows x 1 columns]


In [14]:
# for i in range(len(edges_ca['source'])):
#     print(i, edges_ca[edges_ca['source'] == i].size)
# print(edges_ca[edges_ca['source'] == 1])